In [1]:
import pickle
import pandas as pd
import numpy as np
import optuna

#     optuna_ex1_hyperparameters_per_dataset.py
#  -> analysis_ex1_hyperparameters.ipynb
#  -> benchmark_ex1_best_hyperparameters.py
#  -> analysis_ex1_hyperparameters_best.ipynb

In [2]:
SETUP = "2D"  # "FG" or "2D"
storage = "sqlite:///optuna_databases/optuna_ex1_parameter_tuning_{}_e3_d2.sqlite3".format(SETUP)  

In [3]:
studies = optuna.study.get_all_study_summaries(storage=storage)

In [4]:
loaded_study = optuna.load_study(study_name=studies[0].study_name, storage=storage)
df = loaded_study.trials_dataframe()
for key, value in loaded_study.system_attrs.items():
    df[key] = value
for study in studies[1:]:
    loaded_study = optuna.load_study(study_name=study.study_name , storage=storage)
    df_new = loaded_study.trials_dataframe()
    for key, value in loaded_study.system_attrs.items():
        df_new[key] = value
    df = pd.concat([df, df_new])
df.reset_index(inplace=True, drop=True)

In [5]:
df.columns

Index(['number', 'value', 'datetime_start', 'datetime_complete', 'duration',
       'params_init_learning_rate', 'params_learning_rate_decay',
       'params_max_depth', 'params_n_experts', 'params_optimization_method',
       'params_use_2_dim_clustering', 'params_use_2_dim_gate_based_on',
       'user_attrs_accuracy_train', 'user_attrs_accuracy_val',
       'user_attrs_std_train', 'user_attrs_std_val', 'state', 'Data X',
       'Data y', 'initialization_method', 'params_alpha', 'params_beta',
       'params_gamma', 'params_mean_precision_prior',
       'params_weight_concentration_prior',
       'params_weight_concentration_prior_type', 'params_weight_cutoff'],
      dtype='object')

In [6]:
df.head(1)

,number,value,datetime_start,datetime_complete,duration,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_optimization_method,...,Data X,Data y,initialization_method,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_concentration_prior_type,params_weight_cutoff
0,0,0.695083,2021-08-19 17:13:01.135369,2021-08-19 17:13:08.850243,0 days 00:00:07.714874,171.105182,0.900467,2,3,least_squares_linear_regression,...,abalone_input.pd,abalone_target.pd,str,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.initialization_method.value_counts()

BGM_init         4500
str              4500
KDTmeans_init    4500
Kmeans_init      4500
Name: initialization_method, dtype: int64

In [8]:
np.unique(df.initialization_method)

array(['BGM_init', 'KDTmeans_init', 'Kmeans_init', 'str'], dtype=object)

In [9]:
np.unique(df["Data X"])

array(['abalone_input.pd', 'banknote_input.pd', 'breast_cancer_input.np',
       'cars_input.pd', 'contraceptive_input.pd', 'generated6_input.np',
       'iris_input.pd', 'steel_input.pd', 'students_input.pd'],
      dtype=object)

In [10]:
top = 10
df_best = pd.DataFrame()
for dataset in np.unique(df["Data X"]):
    df_new = df[df["Data X"] == dataset].sort_values(by=["user_attrs_accuracy_val"], ascending=False).head(top)
    df_best = pd.concat([df_best, df_new])
df_best.reset_index(inplace=True, drop=True)


In [11]:
df_best.shape

(90, 27)

In [12]:
df_best.head(2)

,number,value,datetime_start,datetime_complete,duration,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_optimization_method,...,Data X,Data y,initialization_method,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_concentration_prior_type,params_weight_cutoff
0,492,0.741379,2021-08-19 21:44:59.707298,2021-08-19 21:45:08.024373,0 days 00:00:08.317075,243.639280,0.999893,2,3,ridge_regression,...,abalone_input.pd,abalone_target.pd,BGM_init,NaN,NaN,NaN,0.834508,0.4305,dirichlet_process,0.0
1,153,0.739783,2021-08-19 17:32:40.891813,2021-08-19 17:32:48.194964,0 days 00:00:07.303151,287.830823,0.973771,2,3,ridge_regression,...,abalone_input.pd,abalone_target.pd,str,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
pickle.dump(df_best, open("dataframes/ex1_df_top10_hyperparameters_per_dataset_{}_e3_d2.pd".format(SETUP), "wb"))

In [14]:
df.groupby(["Data X"]).mean()

,number,value,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_use_2_dim_clustering,user_attrs_accuracy_train,user_attrs_accuracy_val,user_attrs_std_train,user_attrs_std_val,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_cutoff
Data X,,,,,,,,,,,,,,,,,
abalone_input.pd,249.5,0.712807,232.943860,0.968983,2.0,3.0,True,0.734923,0.712807,0.006760,0.013505,4.469043,0.303451,0.112588,0.583625,0.532108,0.0
banknote_input.pd,249.5,0.984657,194.066541,0.951025,2.0,3.0,True,0.992383,0.984657,0.003566,0.007566,8.122368,0.276558,0.088123,0.568022,0.668796,0.0
breast_cancer_input.np,249.5,0.916256,147.653107,0.879173,2.0,3.0,True,0.963544,0.916256,0.006859,0.021538,6.066354,0.227707,0.284858,0.736271,0.573872,0.0
cars_input.pd,249.5,0.806667,190.200984,0.908500,2.0,3.0,True,0.815370,0.806667,0.007999,0.016961,3.234971,0.297726,0.181250,0.572345,0.428389,0.0
contraceptive_input.pd,249.5,0.537740,229.326490,0.960812,2.0,3.0,True,0.574687,0.537740,0.012078,0.025687,7.891574,0.109973,0.200073,0.433622,0.757395,0.0
generated6_input.np,249.5,0.938736,215.589781,0.963189,2.0,3.0,True,0.944929,0.938736,0.011787,0.014546,1.566131,0.281609,0.099778,0.820436,0.795478,0.0
iris_input.pd,249.5,0.942170,146.716416,0.888464,2.0,3.0,True,0.991085,0.942170,0.006708,0.036399,2.748551,0.295658,0.190604,0.693596,0.331201,0.0
steel_input.pd,249.5,0.610427,177.421932,0.968721,2.0,3.0,True,0.642119,0.610427,0.026277,0.034523,6.346828,0.061826,0.107724,0.419322,0.442619,0.0
students_input.pd,249.5,0.488923,142.889303,0.877320,2.0,3.0,True,0.571311,0.488923,0.017806,0.053371,3.808392,0.244557,0.262980,0.590211,0.740251,0.0


In [15]:
df_best.groupby(["Data X"]).mean()

,number,value,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_use_2_dim_clustering,user_attrs_accuracy_train,user_attrs_accuracy_val,user_attrs_std_train,user_attrs_std_val,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_cutoff
Data X,,,,,,,,,,,,,,,,,
abalone_input.pd,320.5,0.737516,272.319911,0.979849,2.0,3.0,True,0.748031,0.737516,0.006256,0.011343,NaN,NaN,NaN,0.834508,0.430500,0.0
banknote_input.pd,311.0,0.999030,208.351448,0.991758,2.0,3.0,True,0.999773,0.999030,0.000284,0.001680,8.638778,0.270554,0.069166,0.534683,0.617863,0.0
breast_cancer_input.np,176.5,0.943220,122.086597,0.875932,2.0,3.0,True,0.964557,0.943220,0.006651,0.017600,7.747738,0.406913,0.398172,0.658638,0.521160,0.0
cars_input.pd,197.7,0.823148,178.943734,0.867868,2.0,3.0,True,0.826620,0.823148,0.006811,0.018212,NaN,NaN,NaN,0.421252,0.177250,0.0
contraceptive_input.pd,345.4,0.575815,270.384650,0.987600,2.0,3.0,True,0.588527,0.575815,0.007783,0.021618,NaN,NaN,NaN,NaN,NaN,NaN
generated6_input.np,260.7,0.974400,236.885537,0.974267,2.0,3.0,True,0.975671,0.974400,0.001178,0.003141,0.580603,0.307961,0.144244,0.882077,0.944650,0.0
iris_input.pd,293.2,0.975893,152.981868,0.899464,2.0,3.0,True,0.989583,0.975893,0.007121,0.023149,NaN,NaN,NaN,0.925363,0.317167,0.0
steel_input.pd,278.7,0.668590,214.372431,0.991114,2.0,3.0,True,0.685085,0.668590,0.012164,0.022889,6.439211,0.022671,0.088187,0.289817,0.393146,0.0
students_input.pd,300.2,0.557334,143.007234,0.862218,2.0,3.0,True,0.578288,0.557334,0.023297,0.036561,3.165235,0.270902,0.137357,0.582911,0.827739,0.0


In [16]:
df["params_use_2_dim_gate_based_on"].value_counts(dropna=False)

feature_importance_lda_max         4729
feature_importance_lr_max          3764
feature_importance_lr              3372
feature_importance_xgb             1945
feature_importance                 1901
feature_importance_pca_loadings    1406
PCA                                 883
Name: params_use_2_dim_gate_based_on, dtype: int64

In [24]:
df_best["params_optimization_method"].value_counts(dropna=False)

least_squares_linear_regression    50
ridge_regression                   28
lasso_regression                   12
Name: params_optimization_method, dtype: int64

In [25]:
df_best.groupby(["params_optimization_method"]).mean()

,number,value,params_init_learning_rate,params_learning_rate_decay,params_max_depth,params_n_experts,params_use_2_dim_clustering,user_attrs_accuracy_train,user_attrs_accuracy_val,user_attrs_std_train,user_attrs_std_val,params_alpha,params_beta,params_gamma,params_mean_precision_prior,params_weight_concentration_prior,params_weight_cutoff
params_optimization_method,,,,,,,,,,,,,,,,,
lasso_regression,207.833333,0.766860,159.781528,0.879338,2.0,3.0,True,0.783937,0.766860,0.014594,0.026993,7.747738,0.406913,0.398172,0.776109,0.574748,0.0
least_squares_linear_regression,273.660000,0.816823,207.213048,0.943893,2.0,3.0,True,0.827081,0.816823,0.007474,0.016228,7.803752,0.271542,0.052193,0.519900,0.441534,0.0
ridge_regression,309.357143,0.803786,204.117978,0.948358,2.0,3.0,True,0.814287,0.803786,0.005950,0.015237,3.014285,0.185712,0.147660,0.726411,0.646025,0.0


In [26]:
df_best.columns

Index(['number', 'value', 'datetime_start', 'datetime_complete', 'duration',
       'params_init_learning_rate', 'params_learning_rate_decay',
       'params_max_depth', 'params_n_experts', 'params_optimization_method',
       'params_use_2_dim_clustering', 'params_use_2_dim_gate_based_on',
       'user_attrs_accuracy_train', 'user_attrs_accuracy_val',
       'user_attrs_std_train', 'user_attrs_std_val', 'state', 'Data X',
       'Data y', 'initialization_method', 'params_alpha', 'params_beta',
       'params_gamma', 'params_mean_precision_prior',
       'params_weight_concentration_prior',
       'params_weight_concentration_prior_type', 'params_weight_cutoff'],
      dtype='object')

In [27]:
df_best["params_weight_concentration_prior_type"].value_counts()

dirichlet_process         22
dirichlet_distribution     6
Name: params_weight_concentration_prior_type, dtype: int64

In [28]:
df_best[df_best["params_weight_concentration_prior_type"] == "dirichlet_distribution"][['params_mean_precision_prior',
       'params_weight_concentration_prior']].mean()

params_mean_precision_prior          0.687982
params_weight_concentration_prior    0.328838
dtype: float64

In [29]:
df_best[df_best["params_weight_concentration_prior_type"] == "dirichlet_distribution"][['params_mean_precision_prior',
       'params_weight_concentration_prior']].median()

params_mean_precision_prior          0.853644
params_weight_concentration_prior    0.336011
dtype: float64

In [30]:
df_best[['params_alpha', 'params_beta','params_gamma']].mean()

params_alpha    5.403417
params_beta     0.242164
params_gamma    0.134524
dtype: float64

In [31]:
df_best[['params_alpha', 'params_beta','params_gamma']].median()

params_alpha    6.439211
params_beta     0.280115
params_gamma    0.088187
dtype: float64